In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
from datetime import datetime
from datetime import timedelta
import matplotlib.pyplot as plt
from google.cloud import bigquery
import os

In [2]:
cred_file = r"C:\Users\V0V01S7\AppData\Roaming\gcloud\legacy_credentials\venkatasaiyashaswi.v@walmart.com\adc.json"
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]=cred_file
def query_bq(QUERY):
    client = bigquery.Client(project='wmt-edw-sandbox')
    query_job = client.query(QUERY) # API request
    rows = query_job.result() # Waits for query to finish
    df = rows.to_dataframe()
    return df

In [3]:
code = """select A.DRVR_USER_ID, 
B.ACTV_START_TS, B.ACTV_DUR_MIN_QTY, B.ACTV_DT, 
from `wmt-edw-prod.WW_GEC_VM.DRVR` A inner join `wmt-edw-prod.WW_GEC_VM.DRVR_APPLN_ACTV` B on A.DRVR_USER_ID = B.DRVR_USER_ID
where A.MARKET_NM = 'Walmart Orlando' AND B.ACTV_NM = "ONLINE"
"""
df = query_bq(code)

In [4]:
df["ACTV_START_TS"] = pd.to_datetime(df["ACTV_START_TS"])
df["ACTV_START_TS"] = df["ACTV_START_TS"] + pd.Timedelta(hours=3)
df.head()

,DRVR_USER_ID,ACTV_START_TS,ACTV_DUR_MIN_QTY,ACTV_DT
0,davidmoreno222@gmail.com,2021-03-21 10:28:37.881,72.300000000,2021-03-21
1,jboadasv@gmail.com,2021-03-21 09:52:00.478,12.500000000,2021-03-21
2,jboadasv@gmail.com,2021-03-21 09:52:00.478,12.500000000,2021-03-21
3,karepaujose1@gmail.com,2021-03-21 10:00:18.325,0.100000000,2021-03-21
4,karepaujose1@gmail.com,2021-03-21 10:00:18.325,0.100000000,2021-03-21


In [5]:
df = df.set_index("ACTV_START_TS",drop=True)
df.head()

,DRVR_USER_ID,ACTV_DUR_MIN_QTY,ACTV_DT
ACTV_START_TS,,,
2021-03-21 10:28:37.881,davidmoreno222@gmail.com,72.300000000,2021-03-21
2021-03-21 09:52:00.478,jboadasv@gmail.com,12.500000000,2021-03-21
2021-03-21 09:52:00.478,jboadasv@gmail.com,12.500000000,2021-03-21
2021-03-21 10:00:18.325,karepaujose1@gmail.com,0.100000000,2021-03-21
2021-03-21 10:00:18.325,karepaujose1@gmail.com,0.100000000,2021-03-21


In [6]:
def create_features(df, label=None):
    """
    Creates time series features from datetime index.
    """
    df = df.copy()
    df['date'] = df.index
    df['hour'] = df['date'].dt.hour
    df['dayofweek'] = df['date'].dt.dayofweek
    df['quarter'] = df['date'].dt.quarter
    df['month'] = df['date'].dt.month
    df['year'] = df['date'].dt.year
    df['dayofyear'] = df['date'].dt.dayofyear
    df['dayofmonth'] = df['date'].dt.day
    df['weekofyear'] = df['date'].dt.isocalendar().week
    
    X = df[['hour','dayofweek','quarter','month','year',
           'dayofyear','dayofmonth','weekofyear']]
    if label:
        y = df[label]
        return X, y
    return X

In [7]:
X, y = create_features(df, label='ACTV_DUR_MIN_QTY')

features_and_target = pd.concat([X, y], axis=1)

In [8]:
features_and_target.head()

,hour,dayofweek,quarter,month,year,dayofyear,dayofmonth,weekofyear,ACTV_DUR_MIN_QTY
ACTV_START_TS,,,,,,,,,
2021-03-21 10:28:37.881,10,6,1,3,2021,80,21,11,72.300000000
2021-03-21 09:52:00.478,9,6,1,3,2021,80,21,11,12.500000000
2021-03-21 09:52:00.478,9,6,1,3,2021,80,21,11,12.500000000
2021-03-21 10:00:18.325,10,6,1,3,2021,80,21,11,0.100000000
2021-03-21 10:00:18.325,10,6,1,3,2021,80,21,11,0.100000000


In [10]:
ts = features_and_target 
ts["session"] = ts["hour"].apply(lambda x: "morning" if 6<=x and x<=11 else pass)
ts["session"] = ts["hour"].apply(lambda x: "afternoon" if 12<=x and x<=16)
ts["session"] = ts["hour"].apply(lambda x: "evening" if 17<=x and x<=21)
ts.head()

SyntaxError: invalid syntax (1229283601.py, line 2)